In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install selenium &> /dev/null
!apt-get update &> /dev/null
!apt install chromium-chromedriver &> /dev/null
!cp /usr/lib/chromium-browser/chromedriver /usr/bin &> /dev/null

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

In [ ]:
import os
import time
import re
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from csv import writer
from selenium import webdriver


def write_to_txt(file_name, content):
    with open(file_name, 'a') as f:
        f.write(content + os.linesep)


# Cuộn trang để load toàn bộ dữ liệu
def scroll_to_end(driver):
    screen_height = driver.execute_script("return window.screen.height;")
    i = 1
    while True:
        driver.execute_script(f'window.scrollTo(0, {screen_height}*{i});')
        i += 1
        time.sleep(1)
        scroll_height = driver.execute_script('return document.body.scrollHeight;')
        if screen_height * i > scroll_height:
            break


def write_data_to_csv(data, save_path='data.csv'):
    with open(save_path, 'a') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow(data)
        f_object.close()


TAG_RE = re.compile(r'<[^>]+>')

# xóa các tag trong string
def remove_tags(text):
    return TAG_RE.sub('', text)


def cleaning_description(text):
    removed_tags_text = remove_tags(text)
    return re.sub('\s{2,}', ' ', removed_tags_text)


def get_chrome_browser():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    return driver

In [ ]:
driver = get_chrome_browser()

In [ ]:
# driver.get('https://tiki.vn/sach-truyen-tieng-viet')
# category_tags = driver.find_elements('xpath', '//div[@id="collapse-category"]//div//div[@class="list-group-item"]//a')

In [ ]:
def normalizing(text):
    return ' '.join([i.strip() for i in text.split('-')]).capitalize()
df_category = pd.read_csv('/content/drive/MyDrive/Ky1nam4/DoAnCuoiKy/Data_final/Gopfile.csv')
df_category['category_'] = df_category['category'].apply(normalizing)
df_category

,category,id,sku,short_url,category_
0,Nhà Sách Tiki/Sách tiếng Việt/Sách Bà mẹ - Em ...,141483251,8070577886994,https://tiki.vn/product-p141483251.html?spid=1...,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...
1,Nhà Sách Tiki/Sách tiếng Việt/Sách Bà mẹ - Em ...,125362334,8806986919286,https://tiki.vn/product-p125362334.html?spid=1...,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...
2,Nhà Sách Tiki/Sách tiếng Việt/Sách Bà mẹ - Em ...,58213080,4467229116190,https://tiki.vn/product-p58213080.html?spid=58...,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...
3,Nhà Sách Tiki/Sách tiếng Việt/Sách Bà mẹ - Em ...,52798281,9856062766740,https://tiki.vn/product-p52798281.html?spid=52...,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...
4,Nhà Sách Tiki/Sách tiếng Việt/Sách Bà mẹ - Em ...,532292,2601852159830,https://tiki.vn/product-p532292.html?spid=253150,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...
...,...,...,...,...,...
35029,Nhà Sách Tiki/Sách tiếng Việt/Sách Y Học,198415104,2756813127447,https://tiki.vn/product-p198415104.html?spid=1...,Nhà sách tiki/sách tiếng việt/sách y học
35030,Nhà Sách Tiki/Sách tiếng Việt/Sách Y Học,174044900,2321483153120,https://tiki.vn/product-p174044900.html?spid=1...,Nhà sách tiki/sách tiếng việt/sách y học
35031,Nhà Sách Tiki/Sách tiếng Việt/Sách Y Học,186233254,4957675911989,https://tiki.vn/product-p186233254.html?spid=1...,Nhà sách tiki/sách tiếng việt/sách y học
35032,Nhà Sách Tiki/Sách tiếng Việt/Sách Y Học,118375094,2575092201040,https://tiki.vn/product-p118375094.html?spid=2...,Nhà sách tiki/sách tiếng việt/sách y học


In [ ]:
def book_mapping(category, url, book_data):
    book_dict = {
        'category':category,
        'url':url,
        'id':None,
        'sku':None,
        'name':None,
        'price':None,
        'rating_average':None,
        'review_count':None,
        'all_time_quantity_sold':None,
        'description':None,
        'specifications':None,
        'seller_id':None,
        'seller_sku':None,
        'seller_name':None,
        'seller_store_id':None
    }

    keys = book_data.keys()
    for attr in book_dict.keys():
        if 'seller' in attr and 'current_seller' in keys:
            if attr == 'seller_store_id':
                book_dict[attr] = book_data['current_seller']['store_id']
            else:
                book_dict[attr] = book_data['current_seller'][attr.split('_')[1]]
            continue
            # sub_keys = book_data['current_seller'].keys()
            # for sub_attr in book_dict[attr].keys():
            #     if sub_attr in sub_keys:
            #         book_dict[attr][sub_attr] = book_data['current_seller'][sub_attr]

        if attr in keys:
            if attr == 'description':
                book_dict[attr] = cleaning_description(book_data[attr])
                continue

            if attr == 'specifications':
                if len(book_data[attr]):
                    book_dict[attr] = book_data[attr][0]
                continue

            book_dict[attr] = book_data[attr]

    return book_dict


def review_mapping(reviews_data, rating_average, before_reviews):
    id_before = [i['id'] for i in before_reviews]

    for review_data in reviews_data:
        review_dict = {
            'id':None,
            'product_id':None,
            'user_id':None,
            'user_name':None,
            'rating':None,
            'content':None,
            'thank_count':None,
            'rating_average':rating_average
        }

        keys = review_data.keys()
        for attr in review_dict.keys():
            if attr == 'user_id':
                if 'created_by' in keys:
                    if 'id' in review_data['created_by'].keys():
                        review_dict[attr] = review_data['created_by']['id']
                continue
            if attr == 'user_name':
                if 'created_by' in keys:
                    if 'full_name' in review_data['created_by'].keys():
                        review_dict[attr] = review_data['created_by']['full_name']
                continue
            if attr in keys:
                review_dict[attr] = review_data[attr]
        if review_dict['id'] in id_before:
            continue
        before_reviews.append(review_dict)
    return before_reviews

In [ ]:
def write_to_csv(book_dict, review_list, book_file_path, review_file_path):
    book_dict_copy = book_dict.copy()
    book_dict_copy['specifications'] = json.dumps(book_dict_copy['specifications'])
    book_to_write = list(book_dict_copy.values())
    with open(book_file_path, 'a') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow(book_to_write)
        f_object.close()

    review_to_write = []
    review_list_copy = review_list.copy()
    for review_item in review_list_copy:
        review_item_to_write = list(review_item.values())
        review_to_write.append(review_item_to_write)
    with open(review_file_path, 'a') as f_object:
        writer_object = writer(f_object)
        writer_object.writerows(review_to_write)
        f_object.close()

In [ ]:
detail_product_api = "https://tiki.vn/api/v2/products/{}"
product_rating_api = "https://tiki.vn/api/v2/reviews?product_id={}&limit={}&page={}"
folder_path = '/content/drive/MyDrive/Ky1nam4/DoAnCuoiKy/Data_final'
book_path = os.path.join(folder_path, 'books.csv')
review_path = os.path.join(folder_path, 'reviews.csv')
log_path = os.path.join(folder_path, 'log.txt')

In [ ]:
error_id = pd.read_csv(log_path, header=None, sep=' - ', engine='python')[1].values
saved_id = pd.read_csv(book_path, header=None)[2].values

In [ ]:
len(saved_id) + len(error_id)

In [ ]:
index_entries = 1
for index in df_category.index:
    index_entries += 1
    print(index_entries)
    id = df_category.iloc[index]['id']
    category = df_category.iloc[index]['category_']
    short_url = df_category.iloc[index]['short_url']

    # if id in saved_id or id in error_id:
    #     continue


    # Book
    url_product = detail_product_api.format(id)
    try:
        # Lấy dữ liệu
        driver.get(url_product)
        content_page = driver.find_elements('xpath', '//body//pre')[0].text
        book_data = json.loads(content_page)

        # Map dữ liệu 
        book_dict = book_mapping(category, short_url, book_data)
    except:
        print('Có lỗi xảy ra khi lấy và map dữ liệu sách!')
        message = f'ERROR Getting And Mapping Book - {id} - {short_url}'
        write_to_txt(log_path, message)
        continue

    # Reviews
    review_list = []
    limit = 20
    url_reviews = product_rating_api.format(id, limit, 0)
    
    try:
        # Lấy dữ liệu
        driver.get(url_reviews)
        content_page = driver.find_elements('xpath', '//body//pre')[0].text
        reviews_data = json.loads(content_page)

        # Lấy điểm đánh giá trung bình
        rating_average = None
        if 'rating_average' in reviews_data.keys():
            rating_average = reviews_data['rating_average']

        # Map dữ liệu
        if 'data' in reviews_data.keys():
            review_list = review_mapping(reviews_data['data'], rating_average, review_list)

        # Lấy thông tin trang
        if 'paging' in reviews_data.keys():
            if 'current_page' in reviews_data['paging'].keys():
                current_page = reviews_data['paging']['current_page']
            if 'last_page' in reviews_data['paging'].keys():
                last_page = reviews_data['paging']['last_page']
    except:
        print('Có lỗi xảy ra khi lấy và map dữ liệu đánh giá người dùng!')
        message = f'ERROR Getting And Mapping Reviews - {id} - {short_url} - 0'
        write_to_txt(log_path, message)
        continue

    # Lấy dữ liệu các trang sau
    try:
        while (current_page + 1) <= last_page:
            url_reviews = product_rating_api.format(id, limit, current_page + 1)
            driver.get(url_reviews)
            content_page = driver.find_elements('xpath', '//body//pre')[0].text
            reviews_data = json.loads(content_page)
            if 'data' in reviews_data.keys():
                review_list = review_mapping(reviews_data['data'], rating_average, review_list)
            current_page += 1
    except:
        print('Có lỗi xảy ra khi lấy và mapping đánh giá người dùng!')
        message = f'ERROR Getting And Mapping Reviews - {id} - {short_url} - {current_page + 1}'
        write_to_txt(log_path, message)
        continue
    
    try:
        # book_path = 'books.csv'
        # review_path = 'reviews.csv'
        # Lưu dữ liệu xuống file
        write_to_csv(book_dict, review_list, book_path, review_path)
    except:
        print('Lỗi khi lưu dữ liệu xuống file!')
        message = f'ERROR Save Data - {id} - {short_url}'
        write_to_txt(log_path, message)

In [ ]:
df_book = pd.read_csv(book_path, header=None)
columns = ['category', 'url', 'id', 'sku', 'name', 'price', 'rating_average', 'review_count',
           'all_time_quantity_sold', 'description', 'specifications',
           'seller_id', 'seller_sku', 'seller_name', 'seller_store_id']
df_book.columns = columns

In [ ]:
for column in df_book.columns:
    print(f'{column} => {df_book[column].isnull().sum()}')

category => 0
url => 1
id => 24
sku => 24
name => 24
price => 24
rating_average => 24
review_count => 24
all_time_quantity_sold => 7032
description => 24
specifications => 28
seller_id => 24
seller_sku => 24
seller_name => 24
seller_store_id => 24


In [ ]:
df_book.head()

,category,url,id,sku,name,price,rating_average,review_count,all_time_quantity_sold,description,specifications,seller_id,seller_sku,seller_name,seller_store_id
0,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...,https://tiki.vn/product-p141483251.html?spid=1...,141483251.0,8.070578e+12,Sách: Thai Giáo Theo Chuyên Gia - 280 Ngày Mỗi...,80500.0,5.0,58.0,508.0,"\nThai Giáo Theo Chuyên Gia, 280 Ngày, Mỗi Ngà...","{""name"": ""Th\u00f4ng tin chung"", ""attributes"":...",36846.0,3.544877e+12,Minh Long Book,36125.0
1,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...,https://tiki.vn/product-p125362334.html?spid=1...,125362334.0,8.806987e+12,Dinh Dưỡng Thai Kỳ (Bà Bầu Ăn Gì Giúp Con Phát...,103950.0,4.8,41.0,486.0,“Dinh dưỡng thai kỳ” giống như một cuốn bách k...,"{""name"": ""Th\u00f4ng tin chung"", ""attributes"":...",1.0,6.946825e+12,Tiki Trading,40395.0
2,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...,https://tiki.vn/product-p58213080.html?spid=58...,58213080.0,4.467229e+12,Montessori – Phương Pháp Giáo Dục Toàn Diện Ch...,95200.0,5.0,16.0,116.0,MONTESSORI – PHƯƠNG PHÁP GIÁO DỤC TOÀN DIỆN CH...,"{""name"": ""Th\u00f4ng tin chung"", ""attributes"":...",53660.0,1.012848e+12,Nhà sách Fahasa,51001.0
3,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...,https://tiki.vn/product-p52798281.html?spid=52...,52798281.0,9.856063e+12,Phương Pháp Giáo Dục Sớm Montessori Cho Trẻ Từ...,79200.0,5.0,15.0,115.0,Bạn đang có con ở độ tuổi 0-3? Đã bao giờ bạn ...,"{""name"": ""Th\u00f4ng tin chung"", ""attributes"":...",53660.0,4.504642e+12,Nhà sách Fahasa,51001.0
4,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...,https://tiki.vn/product-p532292.html?spid=253150,532292.0,2.601852e+12,Học Montessori Để Dạy Trẻ Theo Phương Pháp Mon...,70000.0,4.8,4.0,72.0,Học Montessori Để Dạy Trẻ Theo Phương Pháp Mon...,"{""name"": ""Th\u00f4ng tin chung"", ""attributes"":...",53660.0,4.983970e+12,Nhà sách Fahasa,51001.0


In [ ]:
df_book.to_csv(os.path.join(folder_path, 'final_books.csv'), header=True, index=False)

In [ ]:
df_book.specifications[0]

NameError: ignored

In [ ]:
df_review = pd.read_csv(review_path, header=None)
columns = ['id', 'product_id', 'user_id', 'user_name', 'rating', 'content', 'thank_count', 'rating_average']
df_review.columns = columns

In [ ]:
for column in df_review.columns:
    print(f'{column} => {df_review[column].isnull().sum()}')

id => 0
product_id => 0
user_id => 1044
user_name => 55060
rating => 0
content => 1145856
thank_count => 0
rating_average => 0


In [ ]:
df_review.to_csv(os.path.join(folder_path, 'final_reviews.csv'), header=True, index=False)

In [ ]:
df_review

,id,product_id,user_id,user_name,rating,content,thank_count,rating_average
0,17128305,141483251,27857293.0,nguyễn Trinh,1,Chưa đọc nhưng lúc nhận là thấy khó chịu vì hộ...,0,4.9
1,17889488,141483251,10844450.0,Hong Thu,5,"Sách cũng khá hữu ích, đọc tham khảo cho biết ...",0,4.9
2,16447472,141483251,11384697.0,David Nguyen,5,"Những đánh giá, review chi tiết và những tiêu ...",0,4.9
3,17407937,141483251,5499213.0,Kiên Bùi,5,Sách này cực kỳ hữu ích cho cả bà bầu và bố nh...,0,4.9
4,17245854,141483251,161480.0,Nguyen Ngoc Hoang,5,"sản phẩm giao nhanh, sản phẩm đúng như hình, s...",0,4.9
...,...,...,...,...,...,...,...,...
1833473,14814063,37151279,26666664.0,nguyen thi van da,5,NaN,0,4.7
1833474,5878962,37151279,12266964.0,Mr SoiDen,5,NaN,0,4.7
1833475,5215329,37151279,18551718.0,Nguyên Tường,3,NaN,0,4.7
1833476,12396306,86981323,11976471.0,Giuse Hiền,5,NaN,0,5.0


In [ ]:
books = pd.read_csv('/content/drive/MyDrive/Ky1nam4/DoAnCuoiKy/Data_final/final_book_specification.csv')

In [ ]:
books

,id,category,url,sku,name,price,rating_average,review_count,all_time_quantity_sold,description,...,number_of_page,manufacturer,publication_date,dimensions,translator,edition,note,brand,origin,brand_country
0,45823892,Sách chính trị pháp lý,https://tiki.vn/product-p45823892.html?spid=45...,5035038406451,Bộ Luật Lao Động (sửa đổi) Năm 2019,40000,5.0,262,2269.0,Bộ luật Lao động số 45/2019/QH14 được Quốc hội...,...,160,Nhà Xuất Bản Lao Động,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2717593,Sách chính trị pháp lý,https://tiki.vn/product-p2717593.html?spid=362...,4018236688530,Bình luận khoa học bộ luật dân sự năm 2015,378000,4.6,36,222.0,BÌNH LUẬN KHOA HỌC BỘ LUẬT DÂN SỰ 2015\nTác Gi...,...,1134,Nhà Xuất Bản Công An Nhân Dân,2016-12-01 10:12:34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,165038803,Sách chính trị pháp lý,https://tiki.vn/product-p165038803.html?spid=1...,1693615394870,Giải Mã Quyền Lực Mềm Hoa Kỳ Dưới Thời Barack ...,97200,5.0,2,17.0,"“Nếu ai đó dí súng vào bạn, đòi bạn đưa tiền v...",...,280,Nhà Xuất Bản Thế Giới,2022-02-01 00:00:00,14x20.5cm,NaN,NaN,NaN,NaN,NaN,NaN
3,2732311,Sách chính trị pháp lý,https://tiki.vn/product-p2732311.html?spid=276...,5585897013333,"Pháp luật về môi giới, đầu tư kinh doanh bất đ...",175000,4.8,52,489.0,PHÁP LUẬT VỀ MÔI GIỚI KINH DOANH BẤT ĐỘNG SẢN ...,...,400,Nhà Xuất Bản Lao Động,2018-06-01 16:57:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1645715,Sách chính trị pháp lý,https://tiki.vn/product-p1645715.html?spid=512...,6623091738856,"Bộ Luật Hình Sự Hiện Hành (Bộ Luật Năm 2015, S...",162000,5.0,18,205.0,"Bộ Luật Hình Sự Hiện Hành (Bộ Luật Năm 2015, S...",...,495,Nhà Xuất Bản Chính Trị Quốc Gia Sự Thật,2017-12-03 00:00:00,14.5 x 20.5 cm,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15849,75540322,Sách khoa học kỹ thuật,https://tiki.vn/product-p75540322.html?spid=75...,7597684732356,Chế Tạo Và Lắp Dựng Kết Cấu Thép,72100,4.3,6,23.0,"Để đáp ứng được yêu cầu của thực tiễn, cuốn “C...",...,138,Nhà Xuất Bản Xây Dựng,2020-12-12 00:00:00,17 x 24 cm,Viện Khoa học Công nghệ BXD,NaN,NaN,NaN,NaN,NaN
15850,722053,Sách khoa học kỹ thuật,https://tiki.vn/product-p722053.html?spid=733577,2443445794727,Thực Hành Xử Lý Số Tín Hiệu Với Matlab,180000,4.5,5,23.0,Thực Hành Xử Lý Số Tín Hiệu Với Matlab “Thực h...,...,421,NaN,2008-09-01 00:00:00,16 x 24 cm,NaN,NaN,NaN,NaN,NaN,NaN
15851,189461210,Sách khoa học kỹ thuật,https://tiki.vn/product-p189461210.html?spid=1...,8111425217122,SÁNG TẠO VÀ BẢN CHẤT GIẢI TOÁN TRÊN MÁY TÍNH C...,250000,0.0,0,1.0,Những cuốn sách chất lượng thì thời gian viết...,...,NaN,Nhà Xuất Bản Đại Học Quốc Gia Hà Nội,2022-07-02 09:15:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15852,58444793,Sách khoa học kỹ thuật,https://tiki.vn/product-p58444793.html?spid=10...,5946735774395,CHƯƠNG TRÌNH LUYỆN TẬP THI ĐẤU 3 MÔN PHỐI HỢP ...,166000,0.0,0,6.0,Tác giả: Henry Ash &amp; Marlies Penker\nHiệu ...,...,NaN,Nhà Xuất Bản Thể Thao Và Du Lịch,2020-07-21 11:55:10,NaN,Hạnh Dung,NaN,NaN,NaN,NaN,NaN
